In [1]:
import topogenesis as tg
import numpy as np
import os
import pickle as pk
import random
import pyvista as pv
# TODO: incorporate into pygmo structure

In [2]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [3]:
rdm_lat_path = os.path.relpath("../../data/randomlattice.csv")
# random_envelope = tg.lattice_from_csv(rdm_lat_path)
env_lat_path = os.path.relpath("../../data/macrovoxels.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

random_values = pk.load(open("../../data/randomvalues.pk", "rb"))
random_envelope = reshape_and_store_to_lattice(random_values, envelope_lattice)



In [4]:
# create random configuration
config_rand = [random.randint(0,1) for i in range(125)]

# voxels of the configuration
config_index = [i for i in range(26) if config_rand[i] == 0] # random indices

#config_rand = np.reshape(config_rand,(5,5,5))
#config_rand = np.pad(config_rand, 1) # pad to regain original lattice structure
#config_rand.shape
random_envelope2 = reshape_and_store_to_lattice(config_rand, envelope_lattice)

In [5]:
# create stencil
stencil = tg.create_stencil("von_neumann", 1, 1)

# remove "self" from stencil
stencil.set_index([0,0,0], 0)

# find indices of the neighbours for each voxel 
neighs = random_envelope.find_neighbours(stencil)

# check which voxels are inside the envelope
envlp_voxs = random_envelope.flatten().astype(bool) # True-False values

# map indices to True-False values 
neighs_status = envlp_voxs[neighs].astype(bool)
neighs_status,envlp_voxs
# for voxels inside the envelope
negh_array = np.array(neighs_status[envlp_voxs])

In [6]:
# when the neighbour's status is False that refers to an outer face
# another method to calculate this: len(np.where(negh_array == False)[0])
outer_faces = np.count_nonzero(negh_array==0)

# voxel edge length
l = envelope_lattice.unit[0]

# calculate total surface area of outer faces
A_exterior = (l**2)*outer_faces

# number of in-envelope voxels
in_voxels = len(random_envelope.centroids)

# calculate total volume inclosed in the envelope
V = in_voxels * (l**3)

In [7]:
# edge length of a cube that has the same volume
l_ref = V**(1/3)

# calculate ratio
R_ref = (6*(l_ref**2))/V

In [8]:
RC = (A_exterior/V)/R_ref

RC_cost = abs(RC-1)

In [9]:
# current definition:

def compactness(x, reference_lattice):
    # create the current configuration as a lattice
    curr_envelope = reshape_and_store_to_lattice(x.astype('bool'), reference_lattice)
    # flatten the envelope
    envlp_voxs = curr_envelope.flatten()

    # create stencil
    stencil = tg.create_stencil("von_neumann", 1, 1)
    stencil.set_index([0,0,0], 0)

    # find indices of the neighbours for each voxel 
    neighs = curr_envelope.find_neighbours(stencil)

    # occupation status for the neighbours for each voxel
    neighs_status = envlp_voxs[neighs]

    # for voxels inside the envelope:
    neigh_array = np.array(neighs_status[envlp_voxs.astype("bool")])  

    # when the neighbour's status is False that refers to an outer face
    outer_faces = np.count_nonzero(neigh_array==0)

    # voxel edge length
    l = envelope_lattice.unit[0] # TODO: can we leave this dimension out?

    # calculate total surface area of outer faces
    A_exterior = (l**2)*outer_faces

    # number of in-envelope voxels
    in_voxels = np.count_nonzero(x)

    # calculate total volume inclosed in the envelope
    V = in_voxels * (l**3)

    # edge length of a cube that has the same volume
    l_ref = V**(1/3)

    # calculate ratio
    R_ref = (6*(l_ref**2))/V

    relative_compactness = (A_exterior/V)/R_ref
    return relative_compactness, A_exterior/V, V, A_exterior

In [10]:
diag = np.zeros((5,5,5))
np.fill_diagonal(diag,1)
np.roll(diag,1,0)

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]]])

In [11]:
method2 = compactness(random_values,envelope_lattice)
method3 = compactness(np.array(config_rand),envelope_lattice)
method4 = compactness(diag.flatten(),envelope_lattice)
method3, method4

((1.8662840560779066, 0.18028169014084508, 239625, 43200),
 (1.7099759466766977, 0.4, 16875, 6750))

In [12]:
ar_rand = np.rot90(diag,1)*envelope_lattice
ar_rand2 = np.rot90(diag,3)*envelope_lattice
ar_rand3 = ar_rand + ar_rand2
ar_rand3[2][2][2] = 1
compactness(ar_rand3.flatten(),envelope_lattice)

(1.926003539862875, 0.37037037037037035, 30375, 11250)

In [13]:
p = pv.Plotter(notebook=True)

a = reshape_and_store_to_lattice(np.array(ar_rand.flatten()),envelope_lattice)
ar_rand3.fast_vis(p,color='pink',opacity=1)
p.show(use_ipyvtk=True, screenshot='compactnessdiag.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(204.88887394336027, 84.88887394336027, 144.88887394336027),
 (60.0, -60.0, 0.0),
 (0.0, 0.0, 1.0)]